# Configurações

## Importações

In [34]:
import os

import pandas as pd

from sklearn.preprocessing import StandardScaler
import joblib

## Definição do Dataframe

In [35]:
df = pd.read_csv("../datasets/data_clean.csv")

In [36]:
df.columns

Index(['id', 'name', 'artists', 'duration_ms', 'year', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'speechiness', 'valence',
       'popularity', 'explicit'],
      dtype='object')

# Pré-processamento

## Seleção de Features

In [37]:
df_selecao = df.copy()

In [38]:
features = [
    "year",
    "popularity",
    "acousticness",
    "danceability",
    "energy",
    "valence",
    "explicit",
]

features_normalized = ["acousticness", "danceability", "energy", "valence"]

In [39]:
df_features = df_selecao.copy()

## Normalização

In [40]:
print("=" * 90)
print("🔧 CONSTRUINDO PIPELINE DE NORMALIZAÇÃO")
print("=" * 90)

scaler_reduced = StandardScaler()
scaler_reduced.fit(df_features[features_normalized])

df_features_normalized_reduced_scaled = pd.DataFrame(
    scaler_reduced.transform(df_features[features_normalized]),
    columns=features_normalized,
)

print("✅ Scaler criado e treinado")
print(f"   Features: {features_normalized}")
print(f"   Shape: {df_features_normalized_reduced_scaled.shape}")
print("\n   Média (deve estar próxima a 0):")
print(df_features_normalized_reduced_scaled.mean())
print("\n   Desvio padrão (deve estar próximo a 1):")
print(df_features_normalized_reduced_scaled.std())

print("\n" + "=" * 90)

🔧 CONSTRUINDO PIPELINE DE NORMALIZAÇÃO
✅ Scaler criado e treinado
   Features: ['acousticness', 'danceability', 'energy', 'valence']
   Shape: (169907, 4)

   Média (deve estar próxima a 0):
acousticness   -1.391753e-16
danceability    3.104680e-16
energy          1.498811e-16
valence         1.927043e-16
dtype: float64

   Desvio padrão (deve estar próximo a 1):
acousticness    1.000003
danceability    1.000003
energy          1.000003
valence         1.000003
dtype: float64



## Binarização

In [41]:
print("=" * 90)
print("🔄 APLICANDO BINARIZAÇÃO DE POPULARITY")
print("=" * 90)

POPULARITY_THRESHOLD = 33

print(f"\nThreshold utilizado: {POPULARITY_THRESHOLD}")

print("\n📊 Binarização: df_features['popularity']")
print("-" * 90)

is_popular = (df_features["popularity"] > POPULARITY_THRESHOLD).astype(int)

binary_distribution = is_popular.value_counts().sort_index()
binary_pcts = is_popular.value_counts(normalize=True).sort_index() * 100

print("✅ Binarização aplicada")
print(
    f"   → Classe 0 (≤ {POPULARITY_THRESHOLD}): {binary_distribution[0]} ({binary_pcts[0]:.2f}%)"
)
print(
    f"   → Classe 1 (> {POPULARITY_THRESHOLD}): {binary_distribution[1]} ({binary_pcts[1]:.2f}%)"
)
print(f"   → Total: {len(is_popular)} registros")

df_popularity_binary = pd.DataFrame({"is_popular": is_popular})

print("   → Dataframe final: apenas coluna 'is_popular'")
print(f"   → Shape: {df_popularity_binary.shape}")

print("\n" + "=" * 90)

🔄 APLICANDO BINARIZAÇÃO DE POPULARITY

Threshold utilizado: 33

📊 Binarização: df_features['popularity']
------------------------------------------------------------------------------------------
✅ Binarização aplicada
   → Classe 0 (≤ 33): 84964 (50.01%)
   → Classe 1 (> 33): 84943 (49.99%)
   → Total: 169907 registros
   → Dataframe final: apenas coluna 'is_popular'
   → Shape: (169907, 1)



In [42]:
print("=" * 90)
print("🎵 CRIANDO FEATURES DECADES COM ONE-HOT ENCODING")
print("=" * 90)

# Criar coluna 'decade' a partir de 'year'
df_decades = df_features.copy()
df_decades["decade"] = (df_decades["year"] // 10 * 10).astype(int)

print("\n📊 Extração de décadas")
print("-" * 90)
print(f"   Décadas encontradas no dataset: {sorted(df_decades['decade'].unique())}")
print(f"   Intervalo: {df_decades['decade'].min()} a {df_decades['decade'].max()}")

# Definir todas as décadas de 1920 a 2025
# 1920-1930, 1930-1940, ..., 2020-2030
all_decades = list(range(1920, 2030, 10))

print(f"\n📌 Décadas para One-Hot Encoding: {all_decades}")

# Criar dataframe com One-Hot Encoding
decades_ohe = pd.DataFrame()

for decade in all_decades:
    decade_label = f"{decade}s"
    decades_ohe[decade_label] = (
        (df_decades["decade"] >= decade) & (df_decades["decade"] < decade + 10)
    ).astype(int)

print("\n✅ One-Hot Encoding aplicado")
print(f"   Shape: {decades_ohe.shape}")
print(f"   Features criadas: {list(decades_ohe.columns)}")

print("\n📊 Distribuição de décadas:")
for col in decades_ohe.columns:
    count = decades_ohe[col].sum()
    pct = (count / len(decades_ohe)) * 100
    print(f"   {col}: {int(count)} ({pct:.2f}%)")

print("\n" + "=" * 90)

🎵 CRIANDO FEATURES DECADES COM ONE-HOT ENCODING

📊 Extração de décadas
------------------------------------------------------------------------------------------
   Décadas encontradas no dataset: [np.int64(1920), np.int64(1930), np.int64(1940), np.int64(1950), np.int64(1960), np.int64(1970), np.int64(1980), np.int64(1990), np.int64(2000), np.int64(2010), np.int64(2020)]
   Intervalo: 1920 a 2020

📌 Décadas para One-Hot Encoding: [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]

✅ One-Hot Encoding aplicado
   Shape: (169907, 11)
   Features criadas: ['1920s', '1930s', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s']

📊 Distribuição de décadas:
   1920s: 4446 (2.62%)
   1930s: 8889 (5.23%)
   1940s: 14968 (8.81%)
   1950s: 19950 (11.74%)
   1960s: 20000 (11.77%)
   1970s: 19998 (11.77%)
   1980s: 20000 (11.77%)
   1990s: 20000 (11.77%)
   2000s: 20000 (11.77%)
   2010s: 19900 (11.71%)
   2020s: 1756 (1.03%)



## Mesclagem de dataframes

In [43]:
print("=" * 90)
print("🔗 MESCLANDO DATAFRAMES")
print("=" * 90)

print("\n📊 Criando dataframes finais")
print("-" * 90)

df_temp_full = df.drop(["year", "popularity"], axis=1).copy()
df_temp_full[features_normalized] = df_features_normalized_reduced_scaled[
    features_normalized
].values

df_selecao_normalized_with_decades = pd.concat(
    [df_temp_full, df_popularity_binary, decades_ohe], axis=1
)

print(
    f"\n🔹 df_selecao_normalized_with_decades: {df_selecao_normalized_with_decades.shape}"
)
print("   → Todas as colunas originais (exceto year e popularity)")
print(f"   → Features normalizadas: {features_normalized}")
print("   → Coluna binarizada: is_popular")
print(f"   → Features decades (OHE): {len(decades_ohe.columns)}")

explicit_col = df_features[["explicit"]].reset_index(drop=True)

df_features_normalized_with_decades = pd.concat(
    [
        df_features_normalized_reduced_scaled.reset_index(drop=True),
        df_popularity_binary.reset_index(drop=True),
        explicit_col,
        decades_ohe.reset_index(drop=True)
    ],
    axis=1
)

print(f"\n🔹 df_features_normalized_with_decades: {df_features_normalized_with_decades.shape}")
print(f"   → Features normalizadas: {features_normalized}")
print("   → Coluna binarizada: is_popular")
print("   → Coluna explícita: explicit")
print(f"   → Features decades (OHE): {len(decades_ohe.columns)}")

🔗 MESCLANDO DATAFRAMES

📊 Criando dataframes finais
------------------------------------------------------------------------------------------

🔹 df_selecao_normalized_with_decades: (169907, 23)


   → Todas as colunas originais (exceto year e popularity)
   → Features normalizadas: ['acousticness', 'danceability', 'energy', 'valence']
   → Coluna binarizada: is_popular
   → Features decades (OHE): 11

🔹 df_features_normalized_with_decades: (169907, 17)
   → Features normalizadas: ['acousticness', 'danceability', 'energy', 'valence']
   → Coluna binarizada: is_popular
   → Coluna explícita: explicit
   → Features decades (OHE): 11


# Gerar o dataframe

In [44]:
print("=" * 90)
print("💾 EXPORTANDO DATAFRAMES FINAIS")
print("=" * 90)

os.makedirs("../datasets", exist_ok=True)
os.makedirs("../models", exist_ok=True)

df_selecao_normalized_with_decades.to_csv(
    "../datasets/selecao_normalized_with_decades.csv", index=False, encoding="utf-8"
)
print("\n✅ selecao_normalized_with_decades.csv")
print(
    f"   Shape: {df_selecao_normalized_with_decades.shape[0]} linhas × {df_selecao_normalized_with_decades.shape[1]} colunas"
)

df_features_normalized_with_decades.to_csv(
    "../datasets/features_normalized_with_decades.csv", index=False, encoding="utf-8"
)
print("\n✅ features_normalized_with_decades.csv")
print(
    f"   Shape: {df_features_normalized_with_decades.shape[0]} linhas × {df_features_normalized_with_decades.shape[1]} colunas"
)

joblib.dump(scaler_reduced, "../models/scaler.joblib")
print("\n✅ scaler.joblib")
print(f"   Scaler salvo com as features: {features_normalized}")

print("\n" + "=" * 90)

💾 EXPORTANDO DATAFRAMES FINAIS



✅ selecao_normalized_with_decades.csv
   Shape: 169907 linhas × 23 colunas

✅ features_normalized_with_decades.csv
   Shape: 169907 linhas × 17 colunas

✅ scaler.joblib
   Scaler salvo com as features: ['acousticness', 'danceability', 'energy', 'valence']



# Resultados

Basicamente, normalizei as 4 features de áudio (acousticness, danceability, energy, valence) usando StandardScaler pra padronizar os valores.

Depois transformei a coluna de popularity em algo binário (is_popular), usando 33 como threshold, valor este obtido na análise exploratória.

No final ficou com 12 features: as 4 normalizadas + 1 binária de popularity + 1 binária de explicit + 6 features de década (one-hot encoding).

Exportei tudo em 3 arquivos: 2 CSVs (um completo com todos os dados, outro só com as features) e o scaler em joblib pra usar depois.